In [1]:
import sys
sys.path.append('../src')

import mlflow
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.model_selection import GridSearchCV # type: ignore

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pickle

from functions import *

from data_cleaning import DataCleaning

import os

pd.set_option('display.max_columns', None)

/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [13]:
data = pd.read_csv('../data/daily/2024_09_23.csv')

data1 = pd.read_csv('../data/daily/2024_09_23_2.csv')

data2 = pd.read_csv('../data/daily/2024_09_23_3.csv')

In [6]:
def mass_transformation(data):
    bins = [800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600,
        2800, 3000, 3200, 3400, 3600, 3800, 4000]

    data['distance_bucket'] = pd.cut(data['Distance (y)'], bins = bins,
                               labels=['800-1000', '1000-1200', '1200-1400', '1400-1600', '1600-1800', '1800-2000',
                                       '2000-2200', '2200-2400', '2400-2600', '2600-2800', '2800-3000',
                                       '3000-3200', '3200-3400', '3400-3600', '3600-3800', '3800-4000'])
    
    data = data.rename(columns ={'Weight (pounds)':'weight'})

    data['evening_morning_price'] = (data['Morning Price'] / data['Evening Price'])

    data['breakfast_morning_price'] = (data['Morning Price'] / data['Breakfast Price'])

    data[['Won P/L Before', ]] = data[['Won P/L Before']].map(pd.to_numeric)

    data['SP Odds Decimal1'] = data['SP Odds Decimal']

    return data

def normalization(data):
    
    normalized_df = DataCleaning.normalize_columns(data, ['SP Odds Decimal', 'weight', 
           'Proform Speed Rating', 'Won P/L Before', 'evening_morning_price'])
    
    df1 = normalized_df[['SP Odds Decimal', 'weight', 'Proform Speed Rating', 'Won P/L Before', 'evening_morning_price']]

    df1 = df1.fillna(0)

    return df1

In [8]:
df1 = mass_transformation(data)
df2 = normalization(df1)

In [12]:
with open('../models/logistic_regression_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Use the loaded model to make predictions on new data
y_new_preds = loaded_model.predict(df2)

df1['model_preds'] = y_new_preds

profit_calculation(df1, stake = 1)

Total number of bets: 351
Total return from betting £1.00 on each prediction where model_preds == 1: £169.72
Return per pound invested: £0.48
Model accuracy: 27.07%


## next set

In [14]:
df2 = mass_transformation(data1)
df3 = normalization(df2)

In [15]:
# Use the loaded model to make predictions on new data
y_new_preds = loaded_model.predict(df3)

df2['model_preds'] = y_new_preds

profit_calculation(df2, stake = 1)

Total number of bets: 171
Total return from betting £1.00 on each prediction where model_preds == 1: £120.66
Return per pound invested: £0.71
Model accuracy: 38.60%


## next set

In [17]:
df22 = mass_transformation(data2)
df33 = normalization(df22)

In [23]:


# Use the loaded model to make predictions on new data
y_new_preds = loaded_model.predict(df33)

df22['model_preds'] = y_new_preds

profit_calculation(df22, stake = 2)

Total number of bets: 1094
Total return from betting £2.00 on each prediction where model_preds == 1: £1919.48
Return per pound invested: £0.88
Model accuracy: 32.91%
